In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


In [3]:
page = 0
result = []

while page < 8:
    page = page + 1
    
    response = requests.get('https://www.zrankings.com/ski-resorts/snow?_=1615734995765&page=' + str(page))
    text = BeautifulSoup(response.text, 'html.parser')
    
    snowTable = text.find('div', {'class': 'tableshow table-wrap'})
    rows = snowTable.find_all('tr')
  
    for row in rows[1:]:
        td = row.find_all('td')
        snowData = {}
        snowData['Zone'] = td[2].text.strip()
        resortName = td[1].text.strip()
        snowData['Resort Name'] = resortName.replace("\n","")[:-2]
        snowData['State'] = resortName.replace("\n","")[-2:]
        snowData['Base Elevation (ft)'] = re.findall('\d+', td[6].text.strip())[0]
        snowData['Top Elevation (ft)'] = re.findall('\d+', td[6].text.strip())[1]
        snowData['Snowfall Score'] = td[11].text.strip()
        snowData['Total Snow Score With Preservation'] = td[16].text.strip()

        moreLink = td[17].a["href"]
        response = requests.get('https://www.zrankings.com' + moreLink)
        text = BeautifulSoup(response.text, 'html.parser')
        overallScore = text.find_all('div', {'class': 'overall-compare'})
        
        for score in overallScore:
            div1 = score.find_all('div')
            try:
                snowData['Overall Ranking'] =  re.sub(r"[\n\t\s]*", "", div1[3].find('h4').text) 
            except:
                continue
            try:
                snowData['Regional Ranking'] = div1[6].find('h4').text
            except:
                continue
            try:
                snowData['State Ranking'] = div1[8].find('h4').text 
            except:
                continue
                
        snowStuff = text.find_all('div', {'class': 'snow-stuff'})
        for stuff in snowStuff:
            div2 = stuff.find_all('div')
            try:
                snowData['True Snow Per Year (inches)'] = div2[1]('h3')[0].text[:-1] 
            except:
                continue
            try:
                snowData['Snow Quality Rank'] = div2[3]('h2')[0].text 
            except:
                continue
            try:
                snowData['Dump Potential Rank'] = div2[6]('h2')[0].text
            except:
                continue
            try:
                snowData['MonthsMoreThan90Snow'] = div2[15]('span')[0].text.replace('%', '') 
            except:
                continue
            try:
                snowData['MonthsLessThan30Snow'] = div2[19]('span')[0].text.replace('%', '') 
            except:
                continue
        loca = BeautifulSoup(response.text, 'lxml')
        latlonglist = loca.find_all(attrs={"data-latitude": True, "data-longitude": True})
        for latlong in latlonglist:
            try:
                snowData['latitude'] = latlong['data-latitude']
                snowData['longitude'] = latlong['data-longitude']
            except:
                continue

        sideStats = text.find_all('div', {'class': 'side-stats-2 clearfix'})
        for stats in sideStats:
            span = stats.find_all('span')
            snowData['Acreage'] = span[0].text.replace("acres", "").replace(',', '')
            snowData['Total Runs'] = span[1].text
            snowData['Longest Run (ft)'] = span[2].text.replace('ft', '').replace(',', '')
            snowData['Lifts'] = span[3].text
            snowData['Uphill Max (ppl/hr)'] = span[4].text.replace('ppl/hr', '').replace(',', '')
            snowData['Terrain Parks'] = span[5].text
            snowData['Halfpipes'] = span[7].text

        result.append(snowData)

In [4]:
snowData = pd.DataFrame.from_dict(result)
pd.set_option('display.max_columns', 35)
snowData.head(2)

,Zone,Resort Name,State,Base Elevation (ft),Top Elevation (ft),Snowfall Score,Total Snow Score With Preservation,Overall Ranking,Regional Ranking,State Ranking,True Snow Per Year (inches),Snow Quality Rank,Dump Potential Rank,MonthsMoreThan90Snow,MonthsLessThan30Snow,latitude,longitude,Acreage,Total Runs,Longest Run (ft),Lifts,Uphill Max (ppl/hr),Terrain Parks,Halfpipes
0,Rockies,Alta,UT,8530,10550,96.1,100.0,4,4,2,517,1,2,49.3,1.3,40.589613,-111.638825,2200,50,16800,11,11248,0,0
1,Rockies,Snowbird,UT,7760,11000,90.0,97.5,3,3,1,497,2,5,37.0,3.4,40.583257,-111.655183,2500,85,13200,11,17400,1,0


In [5]:
snowData.to_csv('zRankings.csv')